In [30]:
from openai import OpenAI

class myVLM:
    def __init__(self, model_name="Qwen/Qwen2.5-VL-32B-Instruct"):
        self.client = OpenAI(
            api_key="sk-vzueoszaobqmmtpcbxtycvombxzwbxrbmtzllrdnahoebsui",
            base_url="https://api.siliconflow.cn/v1"
        )
        self.model_name="Pro/Qwen/Qwen2.5-VL-7B-Instruct"
    

    def create_messages(self,image_path,system_prompt,query):
        import base64

        def encode_image(image_path):
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode("utf-8")
        # # Path to your image
        # image_path = "/root/autodl-tmp/test_images/05-03-18-political-release.pdf_page_1.png"

        # Getting the Base64 string
        base64_image = encode_image(image_path)

        msg=[
                {
                    "role": "system",
                    "content": [{"type":"text","text":system_prompt.format(query=query)}]},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/png;base64,{base64_image}"}, 
                        },
                        
                    ],
                }
            ]
        return msg
            
    def invoke(self,image_path,system_prompt,query):
        msg=self.create_messages(image_path,system_prompt,query)

        completion = self.client.chat.completions.create(
            model=self.model_name,
            messages=msg,
        )

        return completion
    

In [31]:
system_prompt="""
你是一个专业的证据抽取助手。请严格根据用户提供的图片资料内容，提取图片中能直接回答查询问题的证据片段，转换为文字陈述，并按照指定格式返回。
    
    查询问题：
    {query}
    
    提取要求：
    1. 只输出与查询直接相关的证据
    2. 适当整理语言，但不要添加额外信息
    3. 以JSON格式返回结果，包含一个名为"evidence_list"的数组
    4. 不要添加任何解释或说明
    5. 如果图片与查询的问题无关，则返回的evidence_lists为空。
    
    正确格式示例：
    {{
        "evidence_list": [
            "证据片段1",
            "证据片段2"
        ]
    }}
    
    """

In [32]:
model=myVLM()
image_path="/root/autodl-tmp/test_images/05-03-18-political-release.pdf_page_1.png"
query="Among the adults conducted the survey on April 25 - May 1 2018, how many adults rated Trump's government ethical standards as poor? "
model.invoke(image_path,system_prompt,query)

ChatCompletion(id='01964e9a0fe83ea543e35dc7ea2b7347', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n    "evidence_list": []\n}\n```', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1745075442, model='Pro/Qwen/Qwen2.5-VL-7B-Instruct', object='chat.completion', service_tier=None, system_fingerprint='', usage=CompletionUsage(completion_tokens=13, prompt_tokens=5035, total_tokens=5048, completion_tokens_details=None, prompt_tokens_details=None))